In [ ]:
# 신경망이 학습할 수 있도록 도와주는 지표 : 손실 함수
# 학습의 목표는 이 손실 함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는 것

# 이미지 분석 : 이미지에서 특징을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법 : 컴퓨터 비전
# 신경망은 이미지를 '있는 그대로' 학습함. 사람의 생각이 개입되지 않음.

# 오버피팅(과적합) : 한 데이터셋에만 지나치게 최적화된 상태 이 과적합을 피하는 것이 머신러닝의 주요한 과제임.


### 오차제곱합
$$ E = \frac1 2  \sum_k (y_k = t_k)^2 $$

In [2]:
# 오차제곱합은 가장 많이 쓰이는 손실 함수이다.
import numpy as np

def sse(y, t):
    return 0.5 * np.sum((y-t) ** 2) 

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sse(np.array(y), np.array(t))

0.09750000000000003

## 교차 엔트로피 오차

$$ E = -\sum_k t_k  \log y_k $$
* $ y_k $는 신경망의 출력, $ t_k $ 는 정답 레이블

In [5]:
# 로그함수에서도 보이다싶이 x가 1일 때 y는 0이 되고, x가 0에 가까워질수록 y의 값은 점점 작아짐.
# 식도 마찬가지로 정답에 해당하는 출력이 커질수록 0에 다가가다가 그 출력이 1일 때 0이 됨.
import numpy as np

def cee(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))  # 아주 작은 값인 delta를 더하는 이유는 np.log()가 0이 되지 않기 위함.

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cee(np.array(y), np.array(t))

0.510825457099338

## 모든 데이터에 대한 교차 엔트로피 오차

$$ E = -\frac1 N \sum_n \sum_k t_nk \log y_nk  $$

* 마지막에 N으로 나누어 정규화함. 이는 '평균 손실 함수'를 구하는 것임.


In [28]:
import sys, os
sys.path.append("/Users/yimjaekyoon/Library/CloudStorage/OneDrive-개인/UnderDL")
import numpy as np
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [36]:
# 이 훈련 데이터에서 무작위로 10장만 빼내려면?
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 0이상 train_size 미만의 수 중에서 무작위로 batch_size 개를 골라냄
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

t_batch

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

### 배치용 교차 엔트로치 오차 구현하기

In [37]:
# 미니배치 같은 배치 데이터를 지원하는 교차 엔트로피 오차(cce)는 ?
batch_size = 10

def cee(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)    
    batch_size = y.shape[0]
    
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [ ]:
# 손실함수를 사용해야 하는 이유 : 신경망 학습에서는 최적의 매개변수를 탐색할 때 손실 함수의 값을 가능한 한 작게 하는 매개 변수 값을 찾음.
# 이 때 매개변수의 미분(기울기)를 계산하고, 그 미분 값을 단서로 매개변수의 값을 서서싷 갱신하는 과정을 반복함.

# 정확도를 지표로 삼아서는 안 되는 이유는 미분 값이 대부분 장소에서 0이 되어 매개변수를 갱신할 수 없기 때문.

